In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
# img = cv2.imread("lena.png")
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# #cv2.imshow("lena", img)

In [ ]:
def gaussian_kernel(size, sigma=1):
    size = int(size) #size taken as 2
    x, y = np.mgrid[-size:size+1, -size:size+1] #5*5 kernel from -2 to 2
    normal = 1 / (2.0 * np.pi * sigma**2)
    g = np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal #x is x coordinate, y is y coordinate in mgrid
    return g

In [ ]:
#  x, y = np.mgrid[-2:3, -2:3]
#  print(x, y)
#  normal = 1 / (2.0 * np.pi * 1**2)
#  print(normal)
#  print(x**2)
#  print(y**2)
#  print((x**2 + y**2))
#  g = np.exp(-((x**2 + y**2) / (2.0*1**2))) * normal
#  print(g)

In [ ]:
from scipy import ndimage
def sobel_filters(img):
    
    Kx = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], np.float32)
    Ky = np.array([[ 1,  2,  1],
                   [ 0,  0,  0],
                   [-1, -2, -1]], np.float32)
    img = img.astype(np.float32)
    Ix = ndimage.filters.convolve(img, Kx) #weighted sum of Kx kernel and pixel of input image img is stored in Ix
    Iy = ndimage.filters.convolve(img, Ky) #weighted sum of Ky kernel and pixel of input image img is stored in Iy

    G = np.hypot(Ix, Iy)
    theta = np.arctan2(Ix, Iy)
    G = (G / G.max()) * 255
    
    return(G, theta)

In [ ]:
def non_max_suppression(img, D):
    M,N = img.shape
    Z = np.zeros((M,N), dtype = np.int8) #Z is output matrix
    angle = D*180/np.pi #converting radian to degree
    angle[angle < 0]+=180 #checking boolean angle < 0 which returns true or false. If true the line is executed
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            try:
                q = 200
                r = 200
                if(0<=angle[i,j]<22.5) or (157.5<=angle[i,j]<180): #angle 0
                    q = img[i, j+1]
                    r = img[i, j-1]
                elif(22.5<=angle[i,j]<67.5): #angle 45
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                elif(67.5<=angle[i,j]<112.5): #angle 90
                    q = img[i-1, j]
                    r = img[i+1, j]
                elif(112.5<=angle[i,j]<157.5): #angle 135
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]
                
                if(img[i, j]>=q) and (img[i, j]>=r):
                    Z[i, j] = img[i, j]
                else:
                    Z[i, j] = 0
            except IndexError as e:
                pass
    return Z

In [ ]:
# def double_threshold(img):
# #     img = cv2.add(image, 150)#increasing pixel value by 100
#     print(img)
#     M,N = img.shape
#     print(img.max())
#     high_threshold = 100
#     low_threshold = high_threshold/4
#     print('high threshold', high_threshold)
#     print('low threshold', low_threshold)
    
    
#     res = np.zeros((M,N), dtype = np.int8)
    
#     #values assigned to strong and weak edges when they are identified
#     weak = np.int32(65)
#     strong = np.int32(255)
    
#     strong_i, strong_j = np.where(img>=high_threshold)
#     zero_i, zero_j = np.where(img<low_threshold)
#     weak_i, weak_j = np.where((img>=low_threshold) & (img<high_threshold))
#     print(img)
        
#     res[strong_i, strong_j] = strong
#     print(res)
#     res[weak_i, weak_j] = weak
#     print(res.tolist())
#     print(res.max())
# #     res[zero_i, zero_j] = 0
    
#     return(res, strong, weak)
     

In [ ]:
def double_threshold(img, lowThreshold = 0.01, highThreshold = 0.1):
    print(img.max())
#     print(img.tolist())
#     high_threshold = img.max() * highThreshold
#     low_threshold = high_threshold * lowThreshold
    high_threshold = img.max() * highThreshold
    low_threshold = high_threshold * lowThreshold
    
    M,N = img.shape
    res = np.zeros((M,N), dtype = np.float32)
    
    weak = np.float32(25)
    strong = np.float32(255)
    
    strong_i, strong_j = np.where(img>=high_threshold)
    weak_i, weak_j = np.where((img<high_threshold) & (img>=low_threshold))
    zero_i, zero_j = np.where(img>low_threshold)
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return(res, weak, strong)

In [ ]:
def hysterisis(img, weak, strong):
    M,N = img.shape
#     print(img.tolist())
    print(weak)
    print(strong)
    for i in range(1, M-1):
        for j in range(1, N-1):
            if(img[i, j]==weak):
                try:
                    if((img[i-1, j-1]==strong) or (img[i, j-1]==strong) or (img[i-1, j+1]==strong) or (img[i-1, j]==strong) or 
                       (img[i, j+1]==strong) or (img[i+1, j-1]== strong) or (img[i+1, j]==strong) or (img[i+1, j+1]==strong)):
                        img[i, j]=strong
                    else:
                        img[i, j]= 0
                except IndexError as e:
                    pass
#     print(img.tolist())
    return img
                    

In [ ]:
img = cv2.imread("lena.png")
if img is None:
    print("Error: Image not found.")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
kernel_size = 3
sigma = 1
kernel = gaussian_kernel(kernel_size, sigma)
filtered_image = cv2.filter2D(img, -1, kernel)
cv2.imwrite('filtered_image.jpg', filtered_image)
cv2.imshow('Filtered Image', filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
G, theta = sobel_filters(filtered_image)
cv2.imwrite('G_normalized.jpg', G.astype(np.uint8))
cv2.imwrite('theta.jpg', theta)
cv2.imshow('Gradient Calculation', G.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow('Gradient angle', theta)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
suppressed_image = non_max_suppression(G.astype(np.uint8), theta)
cv2.imwrite('Suppressed_Image.jpg', suppressed_image)
cv2.imshow('Suppressed Image', suppressed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
thresholded_image, weak_threshold, strong_threshold = double_threshold(suppressed_image)
print(weak_threshold, 'weak  ', strong_threshold, 'strong')
cv2.imwrite('Thresholded.jpg', thresholded_image)
thresholded_image = thresholded_image.astype(np.float32)
cv2.imshow('Thresholded Image', thresholded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
final_image = hysterisis(thresholded_image, weak_threshold, strong_threshold)
cv2.imwrite("final_image.jpg", final_image)
final_image = final_image.astype(np.float32)
cv2.imshow('Final Image', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()